In [1]:
import os
import sys
import pandas as pd
import requests
from dotenv import load_dotenv
import time

# Load environment variables from the .env file
load_dotenv()

BASE_URL = "https://api.ravelry.com"

# The os.getenv() calls will now find the variables loaded from your .env file
RAVELRY_ACCESS_KEY = os.getenv('RAVELRY_ACCESS_KEY')
RAVELRY_PERSONAL_KEY = os.getenv('RAVELRY_PERSONAL_KEY')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# --- ADD THIS DEBUGGING CODE ---
print(f"Access Key Loaded: {RAVELRY_ACCESS_KEY}")
print(f"Personal Key Loaded: {RAVELRY_PERSONAL_KEY}")

Access Key Loaded: read-d4086974ad193fe02828dd97c21b9560
Personal Key Loaded: Eq5JjrVDcMu4Ji01Y2aQ9bMh4gtUpr1JoSYsG7Ri


Writing a function to test grabbing different attributes that may be useful in analysis:

In [3]:
def search_patterns(query, max_pages=100):
    """
    Searches for patterns on Ravelry with a specific query,
    handling pagination to retrieve a large number of results.

    Args:
        query (str): The search term for patterns (e.g., "sweater").
        max_pages (int): The maximum number of pages to fetch. 
                         Set to None to fetch all pages.
    
    Returns:
        pd.DataFrame: A DataFrame of patterns that match the criteria.
    """
    endpoint = f"{BASE_URL}/patterns/search.json"
    all_patterns = []
    page = 1
    
    print(f"Starting pattern search for: '{query}'")

    while True:
        # Parameters for the API request, including the current page
        params = {
            "query": query,
            "page_size": 100,
            "page": page
        }

        try:
            print(f"Fetching page {page}...")
            response = requests.get(endpoint, auth=(RAVELRY_ACCESS_KEY, RAVELRY_PERSONAL_KEY), params=params)
            response.raise_for_status()

            data = response.json()
            patterns_on_page = data.get('patterns', [])
            
            if not patterns_on_page:
                print("No more patterns found. Ending search.")
                break  # Exit the loop if a page is empty

            all_patterns.extend(patterns_on_page)
            
            # Check paginator to see if we are on the last page
            paginator = data.get('paginator', {})
            if paginator.get('last_page') == page:
                print("Reached the last page of results.")
                break

            # Check if we have reached the user-defined max_pages limit
            if max_pages is not None and page >= max_pages:
                print(f"Reached max_pages limit of {max_pages}.")
                break
                
            page += 1
            
            # --- RATE LIMITING ---
            # Wait for 1 second before the next request to be respectful to the API
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None
    
    # Process all collected patterns into a DataFrame
    patterns_data = []
    for pattern in all_patterns:
        patterns_data.append({
            'Name': pattern.get('name'),
            'Designer': pattern.get('designer', {}).get('name'),
            'ID': pattern.get('id'),
            'URL': f"https://www.ravelry.com/patterns/library/{pattern.get('permalink')}",
            'Free': pattern.get('free')
        })

    print(f"Total patterns collected: {len(patterns_data)}")
    return pd.DataFrame(patterns_data)

In [4]:
sweaters = search_patterns("sweater")

Starting pattern search for: 'sweater'
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Fetching page 16...
Fetching page 17...
Fetching page 18...
Fetching page 19...
Fetching page 20...
Fetching page 21...
Fetching page 22...
Fetching page 23...
Fetching page 24...
Fetching page 25...
Fetching page 26...
Fetching page 27...
Fetching page 28...
Fetching page 29...
Fetching page 30...
Fetching page 31...
Fetching page 32...
Fetching page 33...
Fetching page 34...
Fetching page 35...
Fetching page 36...
Fetching page 37...
Fetching page 38...
Fetching page 39...
Fetching page 40...
Fetching page 41...
Fetching page 42...
Fetching page 43...
Fetching page 44...
Fetching page 45...
Fetching page 46...
Fetching page 47...
Fetching page 48...
Fetching p

In [5]:
sweaters.to_csv("sweaters.csv", index=False)